<h3> Run sheet for end user notebooks code refactoring..

In [16]:
import os, re, fnmatch
import pandas as pd
from helper_functions import *

In [17]:
env = "dev_"

In [18]:
# os.mkdir(f"./user_notebooks/{env.strip('_')}/zips/")
# os.mkdir(f"./user_notebooks/{env.strip('_')}/notebooks/")


In [19]:
#prep folders for pulling end user notebooks
try:    
    os.mkdir(f"./user_notebooks/{env.strip('_')}/")
except Exception as e:
    print (e)

[WinError 183] Cannot create a file when that file already exists: './user_notebooks/dev/'


Update the parameter to correct env in "pull_enduser_files.py" and execute the script to extract all the files from end user workspace folders.

In [20]:
def find_replace(directory, filename_filter, find, replace, filePattern="*") -> None:
    """traverses through the given directory and using given conditions and pattern, does a find + replace"""
    for path, dirs, files in os.walk(os.path.abspath(directory)):
        for filename in fnmatch.filter(files, filePattern):
            if filename_filter in filename:
                filepath = os.path.join(path, filename)
                try:
                    with open(filepath, 'r', encoding='utf-8') as f:
                        s = f.read()
                    if find in s:
                        print (f'{path}/{filename}')
                    s = s.replace(find, replace)
                    with open(filepath, "w") as f:
                        f.write(s)
                except Exception as e:
                    print (f'An issue occurred; likely when reading the file. File : {path}/{filename}')

In [21]:
def get_raw_strings_to_replace(full_path):
    global strings_to_replace
    pattern = re.compile(r"\braw\.\b")
    try:
        for filename in os.listdir(full_path):
            if filename.endswith(".py") or filename.endswith(".sql"):
                with open(os.path.join(full_path, filename), "r") as f:
                    for line in f:
                        for word in line.split():
                            if pattern.search(word): 
                                if (len(word.split("."))) == 2:
                                    strings_to_replace.append( re.sub( r'[)"](.*)', '', (re.sub('[^A-Za-z0-9.]+_', '', word)) ) )
                                else:
                                    print (f"This was formatted poorly and is hence not included in the list:\n {word}\n")
            elif os.path.isdir(os.path.join(full_path, filename)) and 'Trash' not in filename:
                get_raw_strings_to_replace(os.path.join(full_path, filename))
            else:
                pass
    except Exception as e:
        pass

In [22]:
def get_cleansed_strings_to_replace(full_path):
    global strings_to_replace
    pattern = re.compile(r"\bcleansed\.\b")
    try:
        for filename in os.listdir(full_path):
            if filename.endswith(".py") or filename.endswith(".sql"):
                with open(os.path.join(full_path, filename), "r") as f:
                    for line in f:
                        for word in line.split():
                            if pattern.search(word): 
                                print (word)
                                print (full_path, filename)
                                if (len(word.split("."))) == 2:
                                    strings_to_replace.append( re.sub( r'[)"](.*)', '', (re.sub('[^A-Za-z0-9.]+_', '', word)) ) )
                                else:
                                    print (f"This was formatted poorly and is hence not included in the list:\n {word}\n")
            elif os.path.isdir(os.path.join(full_path, filename)) and 'Trash' not in filename:
                get_cleansed_strings_to_replace(os.path.join(full_path, filename))
            else:
                pass
    except Exception as e:
        pass

In [23]:
def get_semantic_strings_to_replace(full_path):
    global strings_to_replace
    pattern = re.compile(r"\bsemantic\.\b")
    try:
        for filename in os.listdir(full_path):
            if filename.endswith(".py") or filename.endswith(".sql"):
                with open(os.path.join(full_path, filename), "r") as f:
                    for line in f:
                        for word in line.split():
                            if pattern.search(word): 
                                if (len(word.split("."))) == 2:
                                    strings_to_replace.append( re.sub( r'[)"](.*)', '', (re.sub('[^A-Za-z0-9.]+_', '', word)) ) )
                                else:
                                    print (f"This was formatted poorly and is hence not included in the list:\n {word}\n")
            elif os.path.isdir(os.path.join(full_path, filename)) and 'Trash' not in filename:
                get_semantic_strings_to_replace(os.path.join(full_path, filename))
            else:
                pass
    except Exception as e:
        pass

In [24]:
def get_curated_strings_to_replace(full_path):
    global strings_to_replace
    pattern = re.compile(r"\bcurated\.\b")
    try:
        for filename in os.listdir(full_path):
            if filename.endswith(".py") or filename.endswith(".sql"):
                with open(os.path.join(full_path, filename), "r") as f:
                    for line in f:
                        for word in line.split():
                            if pattern.search(word): 
                                if (len(word.split("."))) == 2:
                                    strings_to_replace.append( re.sub( r'[)"](.*)', '', (re.sub('[^A-Za-z0-9.]+_', '', word)) ) )
                                else:
                                    print (f"This was formatted poorly and is hence not included in the list:\n {word}\n")
            elif os.path.isdir(os.path.join(full_path, filename)) and 'Trash' not in filename:
                get_curated_strings_to_replace(os.path.join(full_path, filename))
            else:
                pass
    except Exception as e:
        pass

In [25]:
strings_to_replace = []

In [26]:
for item in os.listdir(os.path.join(os.getcwd(), 'user_notebooks', 'dev', 'notebooks')):
    if '.zip' not in item:
        path = os.path.join(os.getcwd(), 'user_notebooks', 'dev', 'notebooks', item)
        # print (path)
        get_cleansed_strings_to_replace(path)
        get_curated_strings_to_replace(path)
        get_raw_strings_to_replace(path)
        get_semantic_strings_to_replace(path)

This was formatted poorly and is hence not included in the list:
 spark.table("curated.bom_weatherforecast").display()



In [33]:
strings_to_replace = list(set(strings_to_replace))

In [34]:
df = pd.DataFrame({'current_namespace': strings_to_replace})
# df.to_csv('table_mapping.csv', header=True, index=False)


In [35]:
# new_df = pd.read_csv('table_mapping.csv', header=0)

In [36]:
df.head()

,current_namespace
0,
1,raw.maximo_classification;
2,'cleansed.maximo_reportadhoc'
3,cleansed.crm_0BP_DEF_ADDRESS_ATTR
4,cleansed.isu_0uc_mtr_doc


In [37]:
df.count()

current_namespace    1329
dtype: int64

In [38]:
get_target_namespace('dev_', 'curated', 'dimstormwaternetwork', excel_path='./uc_scope.xlsx')

Warning! Issue occurred while looking up the future namespace for table: curated.dimstormwaternetwork
Warning! Issue occurred while looking up the future namespace for table: curated.dimstormwaternetwork


{'catalog_name': 'dev_curated',
 'database_name': 'dim',
 'table_name': 'stormwaternetwork',
 'new_namespace': 'dev_curated.dim.stormwaternetwork'}

In [39]:
def trim_namespace(given_namespace):
     return given_namespace.replace('__', '').replace('"', '').replace("'", '').replace("--", '').replace(",", '').replace(";", '')

In [40]:
def update_namespace(given_namespace):
    try:
        db = given_namespace.split('.')[0]
        table = given_namespace.split('.')[1]
        print (db, table)
        namespace_elements = get_target_namespace(env, db, table, excel_path='./uc_scope.xlsx')
        return f"{namespace_elements['catalog_name']}.{namespace_elements['database_name']}.{namespace_elements['table_name']}"
    except Exception as e:
        print (e)
        return None

In [41]:
update_namespace('cleansed.isu_te609t')

cleansed isu_te609t


'dev_cleansed.isu.te609t'

In [42]:
df['trimmed_namespace'] = df['current_namespace'].apply(lambda x: trim_namespace(x))
df['future_namespace'] = df['trimmed_namespace'].apply(lambda x: update_namespace(x))

list index out of range
raw maximo_classification
cleansed maximo_reportadhoc
cleansed crm_0BP_DEF_ADDRESS_ATTR
cleansed isu_0uc_mtr_doc
raw cctv_group_ai_identified_defects
curated meterInstallation
Warning! Issue occurred while looking up the future namespace for table: curated.meterInstallation
Warning! Issue occurred while looking up the future namespace for table: curated.meterInstallation
curated dimContract
Warning! Issue occurred while looking up the future namespace for table: curated.dimContract
Warning! Issue occurred while looking up the future namespace for table: curated.dimContract
raw isu_0UC_REGINST_STR_ATTR
curated dimlocation
Warning! Issue occurred while looking up the future namespace for table: curated.dimlocation
Warning! Issue occurred while looking up the future namespace for table: curated.dimlocation
curated dimwaterNetwork
Warning! Issue occurred while looking up the future namespace for table: curated.dimwaterNetwork
Warning! Issue occurred while looking up

In [22]:
# df.to_csv('./find_replace_mapping.csv', header=1, index=0)

In [ ]:
# get_strings_to_replace("./user_notebooks/dev/o8ts")

In [ ]:
strings_to_replace


In [43]:
df.head()

,current_namespace,trimmed_namespace,future_namespace
0,,,None
1,raw.maximo_classification;,raw.maximo_classification,dev_raw.maximo.classification
2,'cleansed.maximo_reportadhoc',cleansed.maximo_reportadhoc,dev_cleansed.maximo.reportadhoc
3,cleansed.crm_0BP_DEF_ADDRESS_ATTR,cleansed.crm_0BP_DEF_ADDRESS_ATTR,dev_cleansed.crm.0BP_DEF_ADDRESS_ATTR
4,cleansed.isu_0uc_mtr_doc,cleansed.isu_0uc_mtr_doc,dev_cleansed.isu.0uc_mtr_doc


In [ ]:
for item in df.values:
    if item[2] != None:
        print (f"Replacing item '{item[1]}' with '{item[2]}'")
        find_replace(f'./user_notebooks/dev/notebooks/', '', item[1], item[2])
    # print (item[0]) # current_namespace
    # print (item[1]) # trimmed_namespace
    # print (item[2]) # future_namespace

In [ ]:
find_replace('./user_notebooks/', '', 'cleansed.beachwatch_testing', 'curated.dim')

In [ ]:

# find_replace('./user_notebooks', '', 'curated.fact', 'curated.fact.')
# find_replace('./user_notebooks', '', 'curated.maximo', 'curated.userconsumption.')

In [ ]:
directory = "./user_notebooks/prod/ub0/ub0@sydneywater.com.au"

full_path = f"{directory}"
for filename in os.listdir(full_path):
    if filename.endswith(".py"):
        with open(os.path.join(full_path, filename), "r") as f:
            for line in f:
                for word in line.split():
                    if "curated." in word and "curated.fact" not in word and "curated.dim" not in word:
                        print(word)

Once happy with the changes in end user notebooks, push them back into the respective workspace folders by executing push_enduser_files.py. This will push all the updated files back into the workspace user folder (as a zip file). The zip file when received at Databricks workspace, the contents are unzipped. This will also push the original files into a "backup" folder.

In [ ]:
curated.dimProperty -> curated.waterconsumption.property

In [ ]:
def get_new_namespace(namespace):
	if "curated.dim" in namespace:
		return namespace.replace("curated.dim", "trial_curated.dim.")
	elif "curated.fact" in namespace:
		return namespace.replace("curated.fact", "trial_curated.fact.")

print(get_new_namespace('curated.dimProperty'))

find_replace('./user_notebooks/', '', namespace, get_new_namespace(namespace), '*.py')